In [53]:
from datetime import datetime

from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.utils.operators import QubitPauliOperator
from pytket.partition import measurement_reduction, MeasurementBitMap, MeasurementSetup, PauliPartitionStrat
from pytket.backends.backendresult import BackendResult
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit

from scipy.optimize import minimize
from numpy import ndarray
from numpy.random import random_sample
from sympy import Symbol
from functools import partial

import qnexus as qnx
from copy import deepcopy


In [103]:
# qibo's
import qibo
from qibo import gates, hamiltonians, models
from qibo.backends import GlobalBackend
from qibo.models.dbi.double_bracket import (
    DoubleBracketGeneratorType,
    DoubleBracketIteration,
)


# boostvqe's
from boostvqe import ansatze
from boostvqe.plotscripts import plot_gradients, plot_loss
from boostvqe.training_utils import Model, vqe_loss
from boostvqe.utils import (
    DBI_D_MATRIX,
    DBI_ENERGIES,
    DBI_FLUCTUATIONS,
    DBI_STEPS,
    FLUCTUATION_FILE,
    GRADS_FILE,
    HAMILTONIAN_FILE,
    LOSS_FILE,
    SEED,
    TOL,
    apply_dbi_steps,
    create_folder,
    generate_path,
    results_dump,
    rotate_h_with_vqe,
    train_vqe,
)
import numpy as np

# TFIM model

Our goal here is to run VQE on the TFIM model, defined as
$$
H_{\text{TFIM}} = \sum_i X_iX_{i+1} + \sum_i Z_i
$$

By the Central Limit Theorem, the error (standard deviation) of the expectation value $\langle  H_{\text{TFIM}}\rangle$ scales with $1/\sqrt{N_{\text{shots}}}$. In this notebook, we will show

1. The VQE result of the TFIM model on 5-10 qubits.
2. How the error scales with number of shots.
3. How the error scales with the size of the system $L$.

In [21]:
# basics
nqubits = 5
h = 3

In [56]:
# helper functions
def exact_expectation_boost(ham, circ):
    # calculates the exact expectation of hamiltonian given a circuit in qibo
    return ham.expectation(
        ham.backend.execute_circuit(circuit=circ).state())

# Boostvqe

In [80]:
qibo.set_backend("tensorflow")

[Qibo 0.2.11|INFO|2024-10-08 13:29:03]: Using tensorflow backend on /device:CPU:0
INFO:qibo.config:Using tensorflow backend on /device:CPU:0


In [81]:
# build hamiltonian
ham_boost = hamiltonians.TFIM(nqubits=nqubits, h=h, dense=False)
print(ham_boost.matrix)

[Qibo 0.2.11|WARNING|2024-10-08 13:29:07]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


tf.Tensor(
[[-5.+0.j -3.+0.j -3.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [-3.+0.j -1.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [-3.+0.j  0.+0.j -1.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 ...
 [ 0.+0.j  0.+0.j  0.+0.j ... -1.+0.j  0.+0.j -3.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j -1.+0.j -3.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ... -3.+0.j -3.+0.j -5.+0.j]], shape=(32, 32), dtype=complex128)


In [82]:
# build ansatz circuit
nlayer = 1
ansatz_circ = ansatze.hdw_efficient(nqubits, nlayer)
print(ansatz_circ.draw())

q0: ─RY─RZ─o───RY─RZ───o─RY─
q1: ─RY─RZ─Z───RY─RZ─o─|─RY─
q2: ─RY─RZ───o─RY─RZ─Z─|─RY─
q3: ─RY─RZ───Z─RY─RZ───|─RY─
q4: ─RY─RZ─────RY─RZ───Z─RY─


In [83]:
# build zero state
zero_state = ham_boost.backend.zero_state(nqubits)
# initial params
params_len = len(ansatz_circ.get_parameters())
# fix numpy seed to ensure replicability of the experiment
seed = 10
np.random.seed(seed)
initial_params = np.random.uniform(-np.pi, np.pi, params_len)
print(initial_params)
# initial energy
c0 = deepcopy(ansatz_circ)
c0.set_parameters(initial_params)
target_energy = np.real(np.min(np.asarray(ham_boost.eigenvalues())))
print('Target enegry:', target_energy)
print('Initial energy:', exact_expectation_boost(ham_boost, c0))

[ 1.70475788 -3.01120431  0.83973663  1.5632809  -0.00938072 -1.72915367
 -1.89712697  1.63696274 -2.07903793 -2.58653723  1.16465009  2.84875441
 -3.11678496  0.07660625  1.96425543  0.70702213  1.39332975 -1.30768123
  2.62495223  1.34821941  0.26731415 -2.2483119  -0.79582348  1.09411377
 -0.36547294]
Target enegry: -15.422871679540702
Initial energy: tf.Tensor(-2.651967569509104, shape=(), dtype=float64)


In [121]:
niter = 3
# define the qibo loss function
objective_boost = partial(vqe_loss)
# logging hisotry
params_history, loss_history, grads_history, fluctuations = {}, {}, {}, {}
# set optimizer
optimizer = 'sgd'
tol = 1e-2

In [122]:
# train vqe
(
    partial_results,
    partial_params_history,
    partial_loss_history,
    partial_grads_history,
    partial_fluctuations,
    vqe,
) = train_vqe(
    deepcopy(ansatz_circ),
    ham_boost,  # Fixed hamiltonian
    optimizer,
    initial_params,
    tol=tol,
    niterations=1,
    nmessage=1,
    loss=objective_boost,
)

INFO:root:Optimization iteration 0/1
INFO:root:Loss -2.652
INFO:root:Minimize the energy
INFO:root:Optimization iteration 1/1
INFO:root:Loss -2.6775
INFO:root:Optimization iteration 2/1
INFO:root:Loss -2.6958
[Qibo 0.2.11|INFO|2024-10-08 15:15:11]: ite 1 : loss -2.677452
INFO:qibo.config:ite 1 : loss -2.677452
INFO:root:Optimization iteration 3/1
INFO:root:Loss -2.7109
INFO:root:Optimization iteration 4/1
INFO:root:Loss -2.724
INFO:root:Optimization iteration 5/1
INFO:root:Loss -2.7358
INFO:root:Optimization iteration 6/1
INFO:root:Loss -2.7465
INFO:root:Optimization iteration 7/1
INFO:root:Loss -2.7565
INFO:root:Optimization iteration 8/1
INFO:root:Loss -2.7658
INFO:root:Optimization iteration 9/1
INFO:root:Loss -2.7746
INFO:root:Optimization iteration 10/1
INFO:root:Loss -2.783
INFO:root:Optimization iteration 11/1
INFO:root:Loss -2.7909
INFO:root:Optimization iteration 12/1
INFO:root:Loss -2.7985
INFO:root:Optimization iteration 13/1
INFO:root:Loss -2.8059
INFO:root:Optimization ite

KeyboardInterrupt: 

# Quantinuum

In [19]:
qnx.client.auth.login_with_credentials()

INFO:httpx:HTTP Request: POST https://nexus.quantinuum.com/auth/login "HTTP/1.1 200 OK"


✅ Successfully logged in as xiaoyue.li@ntu.edu.sg.


In [20]:
# Connect to project
project_ref = qnx.projects.get_or_create(name="TFIM_experiment(VQE)")
project_ref.df()

# set this in the context
qnx.context.set_active_project(project_ref)

INFO:httpx:HTTP Request: GET https://nexus.quantinuum.com/api/projects/v1beta/meta/count?filter%5Barchived%5D=false&filter%5Btimestamps%5D%5Bcreated%5D%5Bafter%5D=2023-01-01%2000%3A00%3A00&filter%5Bname%5D=TFIM_experiment%28VQE%29 "HTTP/1.1 401 Unauthorized"
INFO:httpx:HTTP Request: POST https://nexus.quantinuum.com/auth/tokens/refresh "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://nexus.quantinuum.com/api/projects/v1beta/meta/count?filter%5Barchived%5D=false&filter%5Btimestamps%5D%5Bcreated%5D%5Bafter%5D=2023-01-01%2000%3A00%3A00&filter%5Bname%5D=TFIM_experiment%28VQE%29 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://nexus.quantinuum.com/api/projects/v1beta?filter%5Barchived%5D=false&filter%5Btimestamps%5D%5Bcreated%5D%5Bafter%5D=2023-01-01%2000%3A00%3A00&filter%5Bname%5D=TFIM_experiment%28VQE%29&page%5Bnumber%5D=0 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://nexus.quantinuum.com/api/projects/v1beta?filter%5Barchived%5D=false&filter%5Btimestamps%5D%5Bcreated%

In [89]:
# helper functions
def create_qubit_pauli_string(nqubits, specify_ls, coef):
    '''
    specify_ls: {index:Pauli.X/Y/Z}
    '''
    term = {}
    specified_ids = list(specify_ls.keys())
    for i in range(nqubits):
        if i in specified_ids:
            term.update({Qubit(i):specify_ls[i]})
        else:
            term.update({Qubit(i):Pauli.I})

    return {QubitPauliString(term):coef}

In [100]:
terms = {}
for i in range(nqubits):
    term_x_i = create_qubit_pauli_string(nqubits, {i: Pauli.X, (i+1)%nqubits: Pauli.X}, 1)
    term_z_i = create_qubit_pauli_string(nqubits, {i: Pauli.Z}, h)
    terms.update(term_x_i)
    terms.update(term_z_i)
ham_quantinuum = QubitPauliOperator(terms)

In [101]:
print(ham_quantinuum)

{(Xq[0], Xq[1], Iq[2], Iq[3], Iq[4]): 1, (Zq[0], Iq[1], Iq[2], Iq[3], Iq[4]): 3, (Iq[0], Xq[1], Xq[2], Iq[3], Iq[4]): 1, (Iq[0], Zq[1], Iq[2], Iq[3], Iq[4]): 3, (Iq[0], Iq[1], Xq[2], Xq[3], Iq[4]): 1, (Iq[0], Iq[1], Zq[2], Iq[3], Iq[4]): 3, (Iq[0], Iq[1], Iq[2], Xq[3], Xq[4]): 1, (Iq[0], Iq[1], Iq[2], Zq[3], Iq[4]): 3, (Xq[0], Iq[1], Iq[2], Iq[3], Xq[4]): 1, (Iq[0], Iq[1], Iq[2], Iq[3], Zq[4]): 3}


In [102]:
# measurement setup
terms = [term for term in ham_quantinuum._dict.keys()]
measurement_setup = measurement_reduction(
    terms, strat=PauliPartitionStrat.CommutingSets
)
for mc in measurement_setup.measurement_circs:
    render_circuit_jupyter(mc)

In [94]:
def compute_expectation_paulistring(
    distribution: dict[tuple[int, ...], float], bitmap: MeasurementBitMap
) -> float:
    '''
    This function assumes that the bitmap is in the correct measurement basis
    and evaluates Pauli operators composed of Pauli.Z and Pauli.I.
    It calculates the expectation by counting the parity of the qubits being
    flipped.
    '''
    value = 0
    for bitstring, probability in distribution.items():
        value += probability * (sum(bitstring[i] for i in bitmap.bits) % 2)
    return ((-1) ** bitmap.invert) * (-2 * value + 1)

In [95]:
def compute_expectation_value_from_results(
    results: list[BackendResult],
    measurement_setup: MeasurementSetup,
    operator: QubitPauliOperator,
) -> float:
    '''
    This function loops with the measurement_setup corresponding to the
    hamiltonian, select the corresponding string_coef, results index and
    calculates the total expectation of the input hamiltonian.
    '''
    energy = 0
    for pauli_string, bitmaps in measurement_setup.results.items():
        string_coeff = operator.get(pauli_string, 0.0)
        if string_coeff != 0:
            for bm in bitmaps:
                index = bm.circ_index
                distribution = results[index].get_distribution()
                value = compute_expectation_paulistring(distribution, bm)
                energy += complex(value * string_coeff).real
    return energy

Let's test the correctness of the above setup by calculating the initial energy of the hamiltonian wrt to the vqe ansatz circuit. Before that, we need to convert the ansatz circuit into pytket.

In [106]:
ansatz_qasm = models.Circuit.to_qasm(ansatz_circ)

In [108]:
import pytket.qasm
ansatz_tket = pytket.qasm.circuit_from_qasm_str(ansatz_qasm)
render_circuit_jupyter(ansatz_tket)

In [118]:
print(ansatz_tket.to_latex_file)

<bound method PyCapsule.to_latex_file of [Ry(0) q[0]; Ry(0) q[1]; Ry(0) q[2]; Ry(0) q[3]; Ry(0) q[4]; Rz(0) q[0]; Rz(0) q[1]; Rz(0) q[2]; Rz(0) q[3]; Rz(0) q[4]; CZ q[0], q[1]; CZ q[2], q[3]; Ry(0) q[4]; Ry(0) q[0]; Ry(0) q[1]; Ry(0) q[2]; Ry(0) q[3]; Rz(0) q[4]; Rz(0) q[0]; Rz(0) q[1]; Rz(0) q[2]; Rz(0) q[3]; CZ q[0], q[4]; CZ q[1], q[2]; Ry(0) q[3]; Ry(0) q[0]; Ry(0) q[1]; Ry(0) q[2]; Ry(0) q[4]; ]>


In [114]:
symbol_dict = {s: p for s, p in zip(ansatz_tket.free_symbols(), initial_params)}
print(symbol_dict)

{}


In [119]:
from pytket import Circuit
from sympy import symbols

a, b, c = symbols("a b c")
circ = Circuit(2)
circ.Rx(1, 0).Rx(b, 1).CX(0, 1).Ry(c, 0).Ry(c, 1)

s_map = {a: 2*a, c: a}  # replacement happens simultaneously, and not recursively
circ.symbol_substitution(s_map)
render_circuit_jupyter(circ)

In [120]:
print(circ.free_symbols())

{a, b}


In [113]:
state_prep_circuit = ansatz_tket.copy()
state_prep_circuit.symbol_substitution(symbol_dict)
render_circuit_jupyter(state_prep_circuit)